# 05. Positional Encoding 구현

## 학습 목표
- Sin/Cos Positional Encoding 구현
- 왜 위치 정보가 필요한지 이해
- Sinusoidal vs Learnable PE 비교

## 관련 면접 질문
- Transformer에 위치 정보가 필요한 이유는?
- Sin/Cos를 사용하는 이유는?
- RoPE와 Sinusoidal PE의 차이는?

---
## Pre-Quiz

### Q1. Attention은 왜 위치 정보를 잃나요?
```
당신의 답: 
```

### Q2. Sin/Cos PE의 장점은?
```
당신의 답: 
```

In [ ]:
import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

---
## 1. Sinusoidal Positional Encoding 구현

### 수식
$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

- pos: 위치 (0, 1, 2, ...)
- i: 차원 인덱스
- 짝수 차원: sin, 홀수 차원: cos

In [ ]:
# =================================================
# TODO: Sinusoidal Positional Encoding 구현
# =================================================
#
# 수식:
#   PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
#   PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
#
# 출력: (1, max_len, d_model) - 미리 계산해서 저장
#
# =================================================

class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        # ============================================
        # 여기에 구현하세요
        # 1. position: (max_len, 1) - 0, 1, 2, ..., max_len-1
        # 2. div_term: 10000^(2i/d_model)
        # 3. PE 계산: 짝수=sin, 홀수=cos
        # 4. register_buffer로 저장 (학습 안 함)
        #
        # 힌트:
        #   position = torch.arange(max_len).unsqueeze(1)
        #   div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        # ============================================
        
        pass  # 이 줄을 지우고 구현
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch, seq_len, d_model)
        Returns:
            output: (batch, seq_len, d_model) with PE added
        """
        # ============================================
        # x + PE[:, :seq_len, :]
        # ============================================
        
        pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_pe():
    batch, seq, d_model = 2, 50, 64
    
    pe = SinusoidalPositionalEncoding(d_model, max_len=100)
    x = torch.zeros(batch, seq, d_model)  # 입력이 0이면 PE만 출력
    
    output = pe(x)
    
    assert output.shape == x.shape, f"Shape 오류: {output.shape}"
    
    # 같은 위치는 항상 같은 PE
    assert torch.allclose(output[0], output[1]), "batch간 PE가 다름!"
    
    print("✅ Positional Encoding 테스트 통과!")
    print(f"   Output shape: {output.shape}")

test_pe()

---
## 2. PE 시각화

In [ ]:
d_model = 64
max_len = 100

pe = SinusoidalPositionalEncoding(d_model, max_len, dropout=0.0)
x = torch.zeros(1, max_len, d_model)
pe_values = pe(x)[0].detach().numpy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap
im = axes[0].imshow(pe_values.T, cmap='RdBu', aspect='auto')
axes[0].set_xlabel('Position')
axes[0].set_ylabel('Dimension')
axes[0].set_title('Positional Encoding Heatmap')
plt.colorbar(im, ax=axes[0])

# 특정 차원의 값
for dim in [0, 1, 10, 20, 30]:
    axes[1].plot(pe_values[:, dim], label=f'dim {dim}')

axes[1].set_xlabel('Position')
axes[1].set_ylabel('Value')
axes[1].set_title('Positional Encoding by Dimension\n낮은 차원: 빠른 주기, 높은 차원: 느린 주기')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 3. 위치 간 유사도

In [ ]:
# 위치 간 코사인 유사도
pe_tensor = torch.tensor(pe_values)
pe_norm = pe_tensor / pe_tensor.norm(dim=-1, keepdim=True)
similarity = pe_norm @ pe_norm.T

plt.figure(figsize=(8, 6))
plt.imshow(similarity.numpy()[:50, :50], cmap='viridis')
plt.colorbar()
plt.xlabel('Position')
plt.ylabel('Position')
plt.title('Position 간 코사인 유사도\n대각선: 자기 자신 (1.0)\n가까운 위치: 높은 유사도')
plt.show()

---
## Post-Quiz

### Q1. 학습된 PE 대신 Sin/Cos를 쓰는 장점은?
```
당신의 답: 
```

### Q2. 10000이라는 base 값의 의미는?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 보기</summary>

### Pre-Quiz

**Q1. Attention이 위치 정보를 잃는 이유**
> - Self-Attention은 순서에 **invariant** (permutation equivariant)
> - 입력 순서를 바꿔도 출력도 같은 순서로 바뀜
> - 문장에서 단어 순서 정보가 필수인데 이를 모름

**Q2. Sin/Cos PE 장점**
> - **외삽 가능**: 학습 때보다 긴 시퀀스도 처리
> - **상대 위치**: PE(pos+k)를 PE(pos)의 선형 함수로 표현 가능
> - 파라미터 없음 (메모리 효율)

---

### Post-Quiz

**Q1. Sin/Cos vs Learned**
> - 외삽 가능 (학습 안 본 긴 시퀀스)
> - 실제로 성능은 비슷함
> - BERT는 learned, GPT-2도 learned 사용

**Q2. 10000의 의미**
> - 주파수 범위 결정
> - 낮은 차원: 짧은 주기 (위치 민감)
> - 높은 차원: 긴 주기 (위치 덜 민감)
> - 10000 = 실험적으로 좋은 값

</details>

---
## 구현 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return self.dropout(x)
```

</details>